In [1]:
from selenium import webdriver

In [142]:
import pickle
from selenium import webdriver

browser = webdriver.Firefox()


def get_cookies():
    browser.get("https://login.aliexpress.com/buyer.htm?return=https%3A%2F%2Fwww.aliexpress.com%2F&random=CEA73DF4D81D4775227F78080B9B6126")
    print('input your username and password in Firefox and hit Submit')
    input('Hit Enter here if you have summited the form: <Enter>')
    cookies = browser.get_cookies()
    pickle.dump(cookies, open("cookies.pickle", "wb"))


def set_cookies():
    browser.get("https://aliexpress.com")
    cookies = pickle.load(open("cookies.pickle", "rb"))
    for cookie in cookies:
        browser.add_cookie(cookie)
    browser.get("https://bestselling.aliexpress.com/en")


if __name__ == '__main__':
    get_cookies()


input your username and password in Firefox and hit Submit
Hit Enter here if you have summited the form: <Enter>


In [144]:
from selenium import webdriver
import pickle
import time


driver = webdriver.Firefox()
driver.get("https://aliexpress.com")
cookies = pickle.load(open("cookies.pickle", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)


def extract_product_urls_from_list_page(list_page_url):
    driver.get(list_page_url)
    time.sleep(5)
    cats = driver.find_elements_by_css_selector('span.title')

    all_links = set()
    for ind, cat in enumerate(cats):
        print(cat.text)
        try:
            cat.click()
        except Exception:
            continue
        if ind == 0:
            items = driver.find_elements_by_class_name('item-desc')
            links = [item.get_attribute('href') for item in items]
        else:
            items = driver.find_elements_by_css_selector('div.title > a')
            links = [item.get_attribute('href') for item in items]
        for link in links:
            all_links.add(link)
        time.sleep(2)
    return all_links


if __name__ == '__main__':
    extract_product_urls_from_list_page('https://sale.aliexpress.com/__pc/bestselling.htm')

TOP SELLING
Women
Men
Electronics
Sports
Health & Beauty
Kids & Baby
Home & Garden
Automotive


In [158]:
import pandas as pd
import json
import pickle
from datetime import datetime
from bs4 import BeautifulSoup
import re
import urllib.request


 # XML parsing


row = []
driver = webdriver.Firefox()
driver.get("https://aliexpress.com")
cookies = pickle.load(open("cookies.pickle", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)


def extract_product_info(product_url):
    global row
    driver.get(product_url)
    content = driver.page_source
    
    #HTML Parser
    soup = BeautifulSoup(content, "html.parser")
    
    #XML Parser
    web = urllib.request.urlopen(product_url)
    soup_xml = BeautifulSoup(web.read(), 'lxml')

    product_id = soup.find('input', {'id': 'hid-product-id'})['value']
    #<input type="hidden" id="hid-product-id" value="32865927655" />
    title = soup.find('h1', {'class': 'product-name'}).text
    #<h1 class="product-name" itemprop="name">Harri Potter Bag Hogwarts PU School Badge Small suitcase Shoulder bag Handbags Halloween Christmas Gifts</h1>
    price = float(soup.find('span', {'id': 'j-sku-price'}).text.split('-')[0])

    if soup.find('span', {'id': 'j-sku-discount-price'}):
        discount_price = float(soup.find('span', {'id': 'j-sku-discount-price'}).text.split('-')[0])
    else:
        discount_price = None

    properties = soup.findAll('li', {'class': 'property-item'})
    attrs_dict = {}
    for item in properties:
        name = item.find('span', {'class': 'propery-title'}).text[:-1]
        val = item.find('span', {'class': 'propery-des'}).text
        attrs_dict[name] = val
    description = json.dumps(attrs_dict)

    stars = float(soup.find('span', {'class': 'percent-num'}).text)
    votes = int(soup.find('span', {'itemprop': 'reviewCount'}).text)
    orders = int(soup.find('span', {'id': 'j-order-num'}).text.split()[0].replace(',', ''))
    wishlists = 0  # int(soup.find('span', {'id': 'j-wishlist-num'}).text.strip()[1:-1].split()[0])

    try:
        shipping_cost = soup.find('span', {'class': 'logistics-cost'}).text
        shipping_company = soup.find('span', {'id': 'j-shipping-company'}).text
    except Exception:
        shipping_cost = ''
        shipping_company = ''
    is_free_shipping = shipping_cost == 'Free Shipping'
    is_epacket = shipping_company == 'ePacket'

    primary_image_url = soup.find('div', {'id': 'magnifier'}).find('img')['src']
    #script = soup.findAll('script', {'type': 'text/javascript'})[7].string
    ############ Javascript parser for image data
    
#     pattern = re.compile('window.runParams.imageBigViewURL')

#     scripts = soup_xml.find_all('script')
#     for script in scripts:
#         if(pattern.match(str(script.string))):
#             data = pattern.match(script.string)
#             image_url_2 = json.loads(data.groups()[0])
#             print(image_url_2)
            
#     scripts = soup_xml.findAll('script')
#     for scrip in scripts:
#         script = json.loads(scrip.string)
#         if(pattern.match(str(script))):
#             data = pattern.match(script)
#             image_url_2 = data.groups()
#             print(image_url_2)

    #Testing- works to some extent. Data is getting read
    
#     data  = soup_xml.find_all("script")[6].string
#     print(data)
#     p = re.compile('window.runParams.imageBigViewURL=[\s\S]*;')
#     m = p.match(data)
#     stocks = json.loads(m.groups(0))
#     for stock in stocks:
#         print(stock)
########################## Extract all additional Images####################

#Testing 2 - Use p.search instead of p.match as match only matches at the beginning of the string.
#https://stackoverflow.com/questions/25478591/python-regular-expression-not-working-as-expected/25478642

#Read the difference about group and groups
#https://www.hackerrank.com/challenges/re-group-groups/problem

#Works well but how to extract each image data?Try groups

#  Ignore   #p = re.compile('window.runParams.imageBigViewURL=(?s).*;')
    p = re.compile('window.runParams.imageBigViewURL=[\s\S]*\];')
    scripts  = soup_xml.find_all("script")
    for script in scripts:
        if(p.search(str(script.string))):    
            data = p.search(str(script.string))
            variable_with_value = data.group(0)
            #print(variable_with_value)
            #print(type(variable_with_value))
            image_url_2 = variable_with_value.split('"')[1]
            image_url_3 = variable_with_value.split('"')[3]
            image_url_4 = variable_with_value.split('"')[5]
            image_url_5 = variable_with_value.split('"')[7]
            image_url_6 = variable_with_value.split('"')[9]
            
  #          stocks = json.loads(data.group(0))
   #         print(stocks)
    

    store_id = soup.find('span', {'class': 'store-number'}).text.split('.')[-1]
    store_name = soup.find('span', {'class': 'shop-name'}).find('a').text
    store_start_date = soup.find('span', {'class': 'store-time'}).find('em').text
    store_start_date = datetime.strptime(store_start_date, '%b %d, %Y')

    if soup.find('span', {'class': 'rank-num'}):
        store_feedback_score = int(soup.find('span', {'class': 'rank-num'}).text)
        store_positive_feedback_rate = float(soup.find('span', {'class': 'positive-percent'}).text[:-1]) * 0.01
    else:
        driver.refresh()
        try:
            store_feedback_score = int(soup.find('span', {'class': 'rank-num'}).text)
            store_positive_feedback_rate = float(soup.find('span', {'class': 'positive-percent'}).text[:-1]) * 0.01
        except Exception:
            store_feedback_score = -1
            store_positive_feedback_rate = -1

    try:
        cats = [item.text for item in soup.find('div', {'class': 'ui-breadcrumb'}).findAll('a')]
        category = '||'.join(cats)
    except Exception:
        category = ''

###########Extract different Options with their price and option image. ALso Extract option Title.

# pattern = re.compile('j-sku-list[\s\S]*\]')

# dl = soup.findAll('dl', {'class': 'p-property-item'})
#     for item in dl:
#         option_title = item.find('dt',{'class' : 'p-item-title'}).text
#         if(pattern.search(str(item.find('ul',{'id' : pattern})))):
#             print("Good")
           
        
        
#         name = item.find('span', {'class': 'propery-title'}).text[:-1]
#         val = item.find('span', {'class': 'propery-des'}).text
#         attrs_dict[name] = val

        
############Assign all values to a pandas dataframe############

#product_data = pd.DataFrame(columns=['product_id','title','description','price','discount_price','stars','votes','wishlists','is_free_shipping','is_epacket','primary_image_url','store_id','store_name','store_start_date','store_feedback_score','store_positive_feedback_rate','category','product_url','image_url_2','image_url_3','image_url_4','image_url_5','image_url_6'])

    product_list = [product_id,title,description,price,discount_price,stars,votes,wishlists,is_free_shipping,is_epacket,primary_image_url,store_id,store_name,store_start_date,store_feedback_score,store_positive_feedback_rate,category,product_url,image_url_2,image_url_3,image_url_4,image_url_5,image_url_6]
    
    row.append(product_list)
    
    print(row)
    
    #Convert to Dataframe after all the appending
    
    df = pd.DataFrame(row,columns=['product_id','title','description','price','discount_price','stars','votes','wishlists','is_free_shipping','is_epacket','primary_image_url','store_id','store_name','store_start_date','store_feedback_score','store_positive_feedback_rate','category','product_url','image_url_2','image_url_3','image_url_4','image_url_5','image_url_6'])
    print(df)  
    
    #Save as CSV
    df.to_csv('C:/Users/Yunus Saleem/Desktop/Research/Aliexpress Scraper/product_data.csv')
    
    #     row = {
#         'product_id': product_id,
#         'title': title,
#         'description': description,
#         'price': price,
#         'discount_price': discount_price,
#         'stars': stars,
#         'votes': votes,
#         'orders': orders,
#         'wishlists': wishlists,
#         'is_free_shipping': is_free_shipping,
#         'is_epacket': is_epacket,
#         'primary_image_url': primary_image_url,
        
#         'store_id': store_id,
#         'store_name': store_name,
#         'store_start_date': store_start_date,
#         'store_feedback_score': store_feedback_score,
#         'store_positive_feedback_rate': store_positive_feedback_rate,
#         'category': category,
#         'product_url': product_url
#     }
    #print(row)  
    #df = pd.DataFrame(row,columns=['product_id','title','description','price','discount_price','stars','votes','wishlists','is_free_shipping','is_epacket','primary_image_url','store_id','store_name','store_start_date','store_feedback_score','store_positive_feedback_rate','category','product_url','image_url_2','image_url_3','image_url_4','image_url_5','image_url_6'])
    #print(df)  
    
    #Save as CSV
    #df.to_csv('C:/Users/Yunus Saleem/Desktop/Research/Aliexpress Scraper/product_data.csv')
    
    #return row
    return row
    


if __name__ == '__main__':
    extract_product_info('https://www.aliexpress.com/item/Harri-Potter-Bag-Hogwarts-PU-School-Badge-Small-suitcase-Shoulder-bag-Handbags-Halloween-Christmas-Gifts/32865927655.html?spm=a2g01.11638294.list_product.2.7a404788a0hIwD&gps-id=5562377&scm=1007.19164.111636.0&scm_id=1007.19164.111636.0&scm-url=1007.19164.111636.0&pvid=6794a351-27a3-413f-bb5e-213a9ac20bec')

# df = pd.DataFrame(row)
# df   

[['32865927655', 'Harri Potter Bag Hogwarts PU School Badge Small suitcase Shoulder bag Handbags Halloween Christmas Gifts', '{"Gender": "Women", "Material": "Metal,Synthetic Leather", "Model Number": "BAG-A", "Characters": "HA"}', 20.41, 14.29, 4.8, 28, 0, True, False, 'https://ae01.alicdn.com/kf/HTB1_TOMl0knBKNjSZKPq6x6OFXaD/Harri-Potter-Bag-Hogwarts-PU-School-Badge-Small-suitcase-Shoulder-bag-Handbags-Halloween-Christmas-Gifts.jpg_640x640.jpg', '2499018', 'Fantasy sky Store', datetime.datetime(2016, 9, 15, 0, 0), -1, -1, "Home||All Categories||Novelty & Special Use||Costumes & Accessories||Women's Costumes||Costume Accessories", 'https://www.aliexpress.com/item/Harri-Potter-Bag-Hogwarts-PU-School-Badge-Small-suitcase-Shoulder-bag-Handbags-Halloween-Christmas-Gifts/32865927655.html?spm=a2g01.11638294.list_product.2.7a404788a0hIwD&gps-id=5562377&scm=1007.19164.111636.0&scm_id=1007.19164.111636.0&scm-url=1007.19164.111636.0&pvid=6794a351-27a3-413f-bb5e-213a9ac20bec', 'https://ae01.alic

In [116]:
type(variable_with_value)

NameError: name 'variable_with_value' is not defined

In [44]:
row

['32865927655',
 'Harri Potter Bag Hogwarts PU School Badge Small suitcase Shoulder bag Handbags Halloween Christmas Gifts',
 '{"Gender": "Women", "Material": "Metal,Synthetic Leather", "Model Number": "BAG-A", "Characters": "HA"}',
 20.41,
 14.29,
 4.7,
 26,
 0,
 True,
 False,
 'https://ae01.alicdn.com/kf/HTB1_TOMl0knBKNjSZKPq6x6OFXaD/Harri-Potter-Bag-Hogwarts-PU-School-Badge-Small-suitcase-Shoulder-bag-Handbags-Halloween-Christmas-Gifts.jpg_640x640.jpg',
 '2499018',
 'Fantasy sky Store',
 datetime.datetime(2016, 9, 15, 0, 0),
 -1,
 -1,
 "Home||All Categories||Novelty & Special Use||Costumes & Accessories||Women's Costumes||Costume Accessories",
 'https://www.aliexpress.com/item/Harri-Potter-Bag-Hogwarts-PU-School-Badge-Small-suitcase-Shoulder-bag-Handbags-Halloween-Christmas-Gifts/32865927655.html?spm=a2g01.11638294.list_product.2.7a404788a0hIwD&gps-id=5562377&scm=1007.19164.111636.0&scm_id=1007.19164.111636.0&scm-url=1007.19164.111636.0&pvid=6794a351-27a3-413f-bb5e-213a9ac20bec']